In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'f:\\Data_Science\\Chicken_disease_classification'

In [27]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [28]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [29]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(PARAMS_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config    

In [30]:
import os 
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [33]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config=config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with the below headers \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    

In [34]:
    try:
        config = ConfigurationManager()
        data_ingestion_config = config.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)
        data_ingestion.download_file()
        data_ingestion.extract_zip_file()
    except Exception as e:
        raise e

[2023-11-10 12:19:27,882: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-10 12:19:27,882: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-10 12:19:27,882: INFO: common: Created directory at: artifacts]
[2023-11-10 12:19:27,882: INFO: common: Created directory at: artifacts/data_ingestion]
[2023-11-10 12:19:42,136: INFO: 4226359801: artifacts/data_ingestion/data.zip download with the below headers 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2A4A:3768:180661B:190D8B0:654E1FAB
Accept-Ranges: bytes
Date: Fri, 10 Nov 2023 12:19:04 GMT
Via: 1.1 varnish
X-Served-By: cache-lhr7387-LHR
X-Cache: 